In [1]:
from datetime import timedelta
import pandas as pd

In [3]:
# Carga de datos con pandas

table_01 = pd.read_csv('..//data//raw//dataset_sample_1.csv')
table_02 = pd.read_csv('..//data//raw//dataset_sample_2.csv')

table_01.head()

,UUID_CLIENTE_CONSUMIDOR,PEDIDO,FECHA_SOLUCION,COD_PRODUCTO,CATEGORIA,UNIDADES_BRUTAS,VENTA_BRUTA_CON_IVA
0,5F333C92C61098CC840A180313615250,39562883,2023-09-28,26605,Jabones,1,10043.0
1,323C3C3B1404F866097F000001615250,39758414,2023-10-14,28308,Cuidado Capilar F,1,23859.0
2,6C2FA988251C4F35BD0A180313615250,39107912,2023-08-23,23610,Carnes Frias,1,20152.0
3,37908B3B6309B1549E7F000001615250,39197173,2023-08-30,18460,Cuidado Capilar F,1,21500.0
4,85C5EF6E09B085614D0A180327615250,36658943,2023-01-18,28485,Cuidado Oral,1,10100.0


In [5]:
table_01["FECHA_SOLUCION"] = pd.to_datetime(table_01['FECHA_SOLUCION'])
table_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231000 entries, 0 to 230999
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   UUID_CLIENTE_CONSUMIDOR  231000 non-null  object        
 1   PEDIDO                   231000 non-null  int64         
 2   FECHA_SOLUCION           231000 non-null  datetime64[ns]
 3   COD_PRODUCTO             231000 non-null  int64         
 4   CATEGORIA                229958 non-null  object        
 5   UNIDADES_BRUTAS          231000 non-null  int64         
 6   VENTA_BRUTA_CON_IVA      231000 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(2)
memory usage: 12.3+ MB


In [7]:
snapshot_date = table_01['FECHA_SOLUCION'].max() + pd.Timedelta(days=1)
snapshot_date

Timestamp('2023-11-17 00:00:00')

In [9]:
rfm = table_01.groupby('UUID_CLIENTE_CONSUMIDOR').agg({
    'FECHA_SOLUCION': lambda x: (snapshot_date - x.max()).days,
    'PEDIDO': 'nunique',
    'VENTA_BRUTA_CON_IVA': 'sum'
}).reset_index().rename(columns={
    'FECHA_SOLUCION':'recency_days', # Cuánto tiempo ha pasado desde la última compra del cliente
    'PEDIDO':'frequency', #  Con qué frecuencia el cliente realiza compras.
    'VENTA_BRUTA_CON_IVA':'monetary' # Cuánto dinero ha gastado el cliente en total.

})

In [11]:
rfm.sort_values(by="frequency", ascending=False)

,UUID_CLIENTE_CONSUMIDOR,recency_days,frequency,monetary
34341,899E06B26BFA75DA5D0A180327615250,1,922,31109906.00
18991,7B607AE9057FD15F050A180326615250,1,537,10702789.00
30706,87C86BDD503A79F1FD0A180326615250,28,358,6312207.00
30987,87E88AC459A59CC6940A180326615250,164,152,2982490.00
30711,87C88198398E86258B0A180327615250,6,117,1736917.00
...,...,...,...,...
10,3232BB14AB7343A8237F000001615250,57,1,45521.00
9,3232B814B311878E097F000001615250,170,1,68816.00
7,3232B5815B6E40AFFA7F000001615250,98,1,39379.00
5,3232B37E44337065D37F000001615250,63,1,5766.13


In [12]:
# Scoring (1-5), 5 = mejor
rfm['r_score'] = pd.qcut(rfm['recency_days'], 5, labels=[5,4,3,2,1]).astype(int)
rfm['f_score'] = pd.qcut(rfm['frequency'].rank(method='first'), 5, labels=[1,2,3,4,5]).astype(int)
rfm['m_score'] = pd.qcut(rfm['monetary'], 5, labels=[1,2,3,4,5]).astype(int)

rfm['RFM_score'] = rfm['r_score'].astype(str) + rfm['f_score'].astype(str) + rfm['m_score'].astype(str)
rfm

,UUID_CLIENTE_CONSUMIDOR,recency_days,frequency,monetary,r_score,f_score,m_score,RFM_score
0,3232A74E9E5AC632B67F000001615250,150,1,51578.00,2,1,3,213
1,3232A86CD5286AA5E17F000001615250,112,2,38038.00,3,3,2,332
2,3232AC8F6D324E487D7F000001615250,34,1,23134.00,4,1,1,411
3,3232AD9F1B20EF57907F000001615250,87,1,35992.00,3,1,2,312
4,3232AECD7274E2EDF37F000001615250,18,8,631619.83,5,5,5,555
...,...,...,...,...,...,...,...,...
37565,8BD62042BC5B10BEA00A180327615250,2,1,223850.68,5,3,5,535
37566,8BD62ADD2E13472D650A180327615250,2,1,45598.72,5,3,2,532
37567,8BD671044AF07D1E000A180326615250,2,1,5872.71,5,3,1,531
37568,8BD679E2CDA71A39230A180327615250,2,1,89790.00,5,3,4,534


In [13]:
# Segments
def segment(row):
    if row['RFM_score'].startswith('5'):
        return 'Cliente Oro'
    if row['r_score'] >=4 and row['f_score']>=4:
        return 'Cliente Plata'
    if row['r_score']<=2 and row['f_score']>=4:
        return 'Cliente Bronce'
    return 'other'

rfm['segment'] = rfm.apply(segment, axis=1)
rfm.head()

,UUID_CLIENTE_CONSUMIDOR,recency_days,frequency,monetary,r_score,f_score,m_score,RFM_score,segment
0,3232A74E9E5AC632B67F000001615250,150,1,51578.00,2,1,3,213,other
1,3232A86CD5286AA5E17F000001615250,112,2,38038.00,3,3,2,332,other
2,3232AC8F6D324E487D7F000001615250,34,1,23134.00,4,1,1,411,other
3,3232AD9F1B20EF57907F000001615250,87,1,35992.00,3,1,2,312,other
4,3232AECD7274E2EDF37F000001615250,18,8,631619.83,5,5,5,555,Cliente Oro
